In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip "drive/MyDrive/audio_dataset.zip"

In [ ]:
!rm -r "audio_dataset/train/Laughter/.ipynb_checkpoints"

In [ ]:
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import os
!pip install pydub
from pydub import AudioSegment

In [ ]:
sounds = ['Laughter', 'car_horn', 'dog_barking', 'drilling', 'Fart', 'Guitar', 'Gunshot_and_gunfire', 'Hi-hat', 'Knock',
          'Shatter', 'siren', 'Snare_drum', 'Splash_and_splatter']

In [ ]:
def conv_m4a(input_file):
    with open(input_file, 'rb') as file:
        header = file.read(20)
        if b'\x4D\x34\x41\x20' in header:
            sound = AudioSegment.from_file(input_file, format='m4a')
            out_file = input_file.split('.')[0] + '.wav'
            os.remove(input_file)
            file_handle = sound.export(out_file, format='wav')
            return out_file

        return input_file

In [ ]:
class conf:
    sampling_rate = 30000
    duration = 2
    hop_length = 347*duration
    fmin = 20
    fmax = sampling_rate // 2
    n_mels = 128
    n_fft = n_mels * 20
    samples = sampling_rate * duration

In [ ]:
def create_spectrogram(audio_file, image_file):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)

    y, sr = librosa.load(audio_file, sr = conf.sampling_rate)

    if len(y) > conf.samples:
        y = y[0:0+conf.samples]
    else:
        padding = conf.samples - len(y)
        offset = padding // 2
        y = np.pad(y, (offset, conf.samples - len(y) - offset), 'constant')

    ms = librosa.feature.melspectrogram(y=y, sr=conf.sampling_rate, n_mels=conf.n_mels,
                                        hop_length=conf.hop_length, n_fft=conf.n_fft,
                                        fmin=conf.fmin, fmax=conf.fmax)
    log_ms = librosa.power_to_db(ms, ref=np.max)
    librosa.display.specshow(log_ms, sr=conf.sampling_rate)
    log_ms = log_ms.astype(np.float32)
    fig.savefig(image_file)
    plt.close(fig)

def create_pngs_from_wavs(input_path, output_path):
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    dir = os.listdir(input_path)

    for i, file in enumerate(dir):
        input_file = os.path.join(input_path, file)
        input_file = conv_m4a(input_file)
        output_file = os.path.join(output_path, file+'.png')
        create_spectrogram(input_file, output_file)
        print(file)

In [ ]:
for sound in sounds:
    input_path = f'audio_dataset/train/{sound}'
    output_path = f'spectrograms/train/{sound}'
    create_pngs_from_wavs(input_path, output_path)
    print('Finished ', sound)

In [ ]:
for sound in sounds:
    input_path = f'audio_dataset/val/{sound}'
    output_path = f'spectrograms/val/{sound}'
    create_pngs_from_wavs(input_path, output_path)
    print('Finished ', sound)

In [ ]:
from keras.preprocessing import image

def load_images_from_path(path, label):
    images = []
    labels = []
    dirs = os.listdir(path)
    for file in dirs:
        img = image.load_img(os.path.join(path, file), target_size=(224, 224))
        img_array = image.img_to_array(img)
        images.append(img_array)
        labels.append(label)
    return images, labels

def load_data(paths, labels):
    images = []
    labels_list = []
    for path, label in zip(paths, labels):
        loaded_images, loaded_labels = load_images_from_path(path, label)
        images.extend(loaded_images)
        labels_list.extend(loaded_labels)
    return np.array(images), np.array(labels_list)

In [ ]:
train_paths = [f'spectrograms/train/{sound}' for sound in sounds]
val_paths = [f'spectrograms/val/{sound}' for sound in sounds]

x_train, y_train = load_data(train_paths, range(len(sounds)))
x_test, y_test = load_data(val_paths, range(len(sounds)))

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)
del y_train
del y_test

In [ ]:
(len(x_test), len(y_test_encoded))

In [ ]:
# x_train = np.array(x_train)
# x_test = np.array(x_test)

In [ ]:
from keras.models import Sequential
import tensorflow as tf
from tensorflow.keras import layers
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, BatchNormalization, Dropout, Input
import keras.regularizers as regularizers
import keras.initializers as initializers

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

def conv_block(input_tensor, num_channels, strides=(1, 1)):
    x = layers.Conv2D(num_channels, (1, 1), strides=strides)(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(num_channels, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(num_channels * 4, (1, 1))(x)
    x = layers.BatchNormalization()(x)

    shortcut = layers.Conv2D(num_channels * 4, (1, 1), strides=strides)(input_tensor)
    shortcut = layers.BatchNormalization()(shortcut)

    x = layers.add([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

def identity_block(input_tensor, num_channels):
    x = layers.Conv2D(num_channels, (1, 1))(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(num_channels, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(num_channels * 4, (1, 1))(x)
    x = layers.BatchNormalization()(x)

    x = layers.add([x, input_tensor])
    x = layers.Activation('relu')(x)
    return x

def resnet50(input_shape=(224, 224, 3), classes=13):
    inputs = tf.keras.Input(shape=input_shape)

    x = layers.ZeroPadding2D((3, 3))(inputs)
    x = layers.Conv2D(64, (7, 7), strides=(2, 2))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.ZeroPadding2D((1, 1))(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 64, strides=(1, 1))
    x = identity_block(x, 64)
    x = identity_block(x, 64)

    x = conv_block(x, 128)
    x = identity_block(x, 128)
    x = identity_block(x, 128)
    x = identity_block(x, 128)

    x = conv_block(x, 256)
    x = identity_block(x, 256)
    x = identity_block(x, 256)
    x = identity_block(x, 256)
    x = identity_block(x, 256)
    x = identity_block(x, 256)

    x = conv_block(x, 512)
    x = identity_block(x, 512)
    x = identity_block(x, 512)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(classes, activation='softmax')(x)

    model = tf.keras.Model(inputs, x)
    return model

In [ ]:
# model = Sequential([
#             Input(shape=(224, 224, 3)),
#             Conv2D(64, (7, 7), strides=(2, 2), padding='same'),
#             BatchNormalization(),
#             Activation('relu'),
#             MaxPooling2D((3, 3), strides=(2, 2), padding='same')
#         ])
# num_channels = 64
# for i in range(3):
#     model.add(res_block(num_channels, conv_first=(i==0)))
#     num_channels *= 2
# for i in range(4):
#     model.add(identity_block(num_channels))
# for i in range(6):
#     model.add(identity_block(num_channels*2))
# for i in range(3):
#     model.add(identity_block(num_channels*4))
# model.add(layers.GlobalAveragePooling2D())
# model.add(Dense(13, activation='softmax'))

In [ ]:
model = resnet50()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
hist = model.fit(x_train, y_train_encoded, validation_data=(x_test, y_test_encoded), batch_size=5, epochs=15)

In [ ]:
acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, '-', label='Training Accuracy')
plt.plot(epochs, val_acc, ':', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.plot()

In [ ]:
prediction = model.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
import seaborn as sns
sns.set_theme()

mat = confusion_matrix(y_test_encoded.argmax(axis=1), prediction.argmax(axis=1))
class_labels = ['background', 'chainsaw', 'engine', 'storm']

sns.heatmap(mat, square=True, annot=True, fmt='d', cbar=False, cmap='Blues',
            xticklabels=sounds,
            yticklabels=sounds)

plt.xlabel('Predicted label')
plt.ylabel('Actual label')

In [ ]:
precision_recall_fscore_support(y_test_encoded.argmax(axis=1), prediction.argmax(axis=1))